# Store

> Provides storage for documents

In [ ]:
#| default_exp store

In [ ]:
#| export
from nanorag.base import Document, abstractmethod, ABC
from nanorag.context import ModelContext
from nanorag.loaders import PDFLoader
from typing import Union, List, Dict, Optional
from uuid import UUID
from collections import defaultdict

In [ ]:
#| export
class BaseDocumentStore(ABC):
    """
    Base class for document storage"""
    def __init__(self, documents : Dict[str, Document] = {}):
        pass
    @abstractmethod
    def __call__(self, db): #Connect to backend and specific collection parameters
        pass
    @abstractmethod    
    def add(self, document: Document):
        pass
    @abstractmethod 
    def ids(self):
        pass
    @abstractmethod    
    def delete(self, ids: Union[List, str]):
        pass
    @abstractmethod
    def get(self, ids: Union[UUID, List]):
        pass
        

In [ ]:
#| export
class DocumentStore(BaseDocumentStore):
    """Key value type document store that store documents by their id in a dictionary.
    Also checks for duplicates via hashing and doesn't admit them. Compatible with both nodes and documents."""
    def __init__(self, documents : Union[List[Document], Document]= []):
        if isinstance(documents, list):
            self.documents = {document.id: document for document in documents}
        elif isinstance(documents, Document):
            self.documents = {documents.id: documents}

    def __call__(self, db): #Connect to backend and specific collection parameters
        pass
        
    def add(self, document: Union[List[Document], Document]) -> str:
        doc_ids = []
        if isinstance(document, list):
            for doc in document:
                self.add(doc)
                doc_ids.append(doc.id)
        else:
            for stored_document in self.documents:
                if self.documents[stored_document].hash == document.hash:
                    return f"You tried to add a duplicate document: {document.hash}"
                elif self.documents[stored_document].id == document.id:
                    self.documents[document.id] = document
                    return f"Document with id {document.id} has been updated"
            self.documents[document.id] =  document
        return f"The following documents have been added: {doc_ids}"

    def ids(self):
        doc_ids = [doc for doc in self.documents]
        return doc_ids

    def delete(self, ids: Union[List, str]):
        deleted_docs = []
        if isinstance(ids, List):
            for id in ids:
                deleted_doc = self.documents.pop(id, None)
                if deleted_doc != None:
                    deleted_docs.append(deleted_doc)
        elif isinstance(ids, UUID):
            deleted_doc = self.documents.pop(ids, None)
            print(f'Ids are: {ids} and theoretically deleted doc is {deleted_doc}')
            if deleted_doc != None:
                deleted_docs.append(deleted_doc)
        return f"The following docs have been deleted {deleted_docs}"
        
    def get(self, ids: Optional[Union[List[UUID], UUID]] = None) -> Optional[Union[Document, List[Document]]]:
        if ids == None:
            ids = self.ids()
            if isinstance(ids, List):
                if len(ids) == 0:
                    return None
        if isinstance(ids, List):
            docs = [self.documents[id] for id in ids if id in self.documents]
            if len(docs) == 0:
                return None
            return docs
        elif isinstance(ids, UUID):
            doc = self.documents.get(ids, None)
            if doc is None:
                return None
            return doc
        return None
    def group_by_source_id(self, source_id = None): #Other type of filters can be added
        grouped_documents = defaultdict(list)
        for doc in self.documents.values():
            if source_id == None:
                grouped_documents[doc.source_id].append(doc)
            elif doc.source_id not in grouped_documents or doc.source_id == source_id:
                grouped_documents[doc.source_id].append(doc)
        if source_id != None:
            return grouped_documents[source_id]
        return dict(grouped_documents)
#NOTE: Could I store both nodes and docs in same store? 

In [ ]:
class PostgresDocumentStore(BaseDocumentStore):
    pass

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

Example insert into DocStore

In [ ]:
#| hide
#| eval: false
loader = PDFLoader('datasets/papers_pdf')
documents = loader.get_documents()
store = DocumentStore(documents)

In [ ]:
#| hide
#| eval: false
documents_2 = loader.get_documents()
store.add(documents_2)
source_id = documents_2[0].source_id
document = store.group_by_source_id(source_id)
document

[Document(id = 053657da-a25f-469e-9a34-cd9cd57a43e2, name = EfﬁcientNet: Rethinking Model Scaling for Convolutional Neural Networks, metadata = {'page': 1, '/Author': 'Mingxing Tan, Quoc V. Le', '/CreationDate': 'D:20200914001802Z', '/Creator': 'LaTeX with hyperref package', '/Keywords': 'Machine Learning, ICML', '/ModDate': 'D:20200914001802Z', '/PTEX.Fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.17 (TeX Live 2016) kpathsea version 6.2.2', '/Producer': 'pdfTeX-1.40.17', '/Subject': 'Proceedings of the International Conference on Machine Learning 2019', '/Title': 'EfficientNet: Rethinking Model Scaling for Convolutional Neural Networks', '/Trapped': '/False'}, source_id = f5973cb0-5023-463f-8f4a-e39c32760227),
 Document(id = 9872d5e1-94c1-41da-bc8b-f239bcfd5967, name = EfﬁcientNet: Rethinking Model Scaling for Convolutional Neural Networks, metadata = {'page': 2, '/Author': 'Mingxing Tan, Quoc V. Le', '/CreationDate': 'D:20200914001802Z', '/Creator': 'LaTeX with hyperref pa